# Day 4: Intro to Grama

- Grama motivation
- Grama elements
- Exploratory model analysis


In [1]:
import grama as gr
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt

# Set figure options
plt.rcParams['figure.figsize'] = [6, 6] # Need square aspect ratio for gradients to appear normal
plt.rcParams['figure.dpi'] = 100 # 200 e.g. is really fine, but slower


## Grama Motivation

---

Idea: Have one computational object that you can use to encapsulate *many* of the assumptions and choices in quantitative modeling.

Some example modeling choices:
- Select the model inputs
- Specify bounds for the inputs
- Simplify the physics
- Fit a distribution for inputs

Grama is a software package to help make model building and analysis more transparent, convenient, and understandable.

Modeling in two phases:
- Model building
- Model analysis


### Model Building

**Model Functions**

Parallel RLC circuit: here are some key performance indicators of the circuit

$$\omega_0 = \sqrt{\frac{1}{LC}}$$

$$Q = \omega_0 RC.$$

**Model Domain**

Decide on a range of values for $R, L, C$ to test.

In [ ]:
## Implement RLC Grama model
md_RLC_det = (
    gr.Model("RLC Circuit")
    ## Add the natural frequency
    >> gr.cp_vec_function(
        fun=lambda df: gr.df_make(
            omega0=np.sqrt(1 / df.L / df.C)
        ),
        var=["L", "C"],
        out=["omega0"],
    )
    ## Add the Q factor
    >> gr.cp_vec_function(
        fun=lambda df: gr.df_make(
            Q=df.omega0 * df.R * df.C
        ),
        name="parallel RLC",
        var=["omega0", "R", "C"],
        out=["Q"]
    )
    ## Set bounds for the input variables
    >> gr.cp_bounds(
        R=(1e-3, 1e0),  # resistance [ohm]
        L=(1e-9, 1e-3), # inductance [H]
        C=(1e-3, 10),   # capacitance [F]
    )
)